# Stimare Q function ottimale
## 1. Quanto tempo ci mette per convergere a Q*, al variare di tau
## 2. Distanza tra Q* e Q_t 
## 3. distanza tra Q_0 e Q_0 appresa tramite i valori di tau
## 4. cercare di integrare i bound trovati

In [1]:
import numpy as np
from TMDP import TMDP
from river_swim import River


In [12]:
gamma = 0.9
river = River(gamma)
tau = 0.9
xi = np.ones(river.nS)*1/river.nS
tmdp = TMDP(river, xi, tau, gamma)


In [13]:
from algorithms import Q_learning
from algorithms import eps_greedy
from algorithms import SARSA

In [11]:
tmdp_0 = TMDP(river, xi, gamma)
s = tmdp_0.reset()
M = 100000
Q = np.zeros((tmdp_0.nS, tmdp_0.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])

Q = Q_learning(tmdp_0, s, a, Q, M)

print(Q)

[[  50.           44.66886669]
 [  46.           42.07680793]
 [  42.4          38.36518522]
 [  39.16         36.41201191]
 [  36.244        35.41385209]
 [  33.56831142 9074.11671237]]


In [15]:

s = tmdp.reset()
M = 100000
Q = np.zeros((tmdp.nS, tmdp.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])

Q = Q_learning(tmdp, s, a, Q, M)

print(Q)

[[50.         44.80073335]
 [46.         43.12186974]
 [42.4        40.24116852]
 [39.16       33.73048561]
 [36.244      33.61959913]
 [32.55990516  0.        ]]


In [11]:
taus = [1 - i*0.1 for i in range(10)]
Qs = {}
xi = np.ones(river.nS)*1/river.nS
gamma = 0.9
river = River(gamma)

for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    s = tmdp.reset()
    M = 5000
    Q = np.zeros((tmdp.nS, tmdp.nA))
    ret = 0

[1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.3999999999999999, 0.29999999999999993, 0.19999999999999996, 0.09999999999999998]
